In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd;
import matplotlib.pyplot as plt;
import seaborn as sns;
from sklearn.impute import SimpleImputer;
from sklearn.compose import ColumnTransformer;
from sklearn.pipeline import Pipeline;
from sklearn.preprocessing import LabelEncoder;
from sklearn.preprocessing import StandardScaler;
from sklearn.preprocessing import MinMaxScaler;
from sklearn.model_selection import train_test_split;
from sklearn.linear_model import LinearRegression ;
from sklearn.linear_model import LogisticRegression;
from sklearn.linear_model import Ridge, Lasso;
from sklearn.metrics import mean_squared_error;
from sklearn.metrics import r2_score;
from sklearn.preprocessing import PolynomialFeatures;
from sklearn.svm import SVR;
from sklearn.svm import SVC;
from sklearn.tree import DecisionTreeClassifier;
from sklearn.tree import DecisionTreeRegressor;
from sklearn.ensemble import RandomForestClassifier;
from sklearn.ensemble import RandomForestRegressor;
from sklearn.neighbors import KNeighborsClassifier;
from sklearn.neighbors import KNeighborsRegressor;
from sklearn.naive_bayes import GaussianNB;
import xgboost as xgb;
from xgboost import XGBClassifier;
from xgboost import XGBRegressor;

import tensorflow as tf
import keras;
from keras_preprocessing import image;
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam;
from keras.callbacks import ModelCheckpoint;
from keras.models import Sequential;
from tensorflow.keras.applications import VGG16;
from tensorflow.keras.applications import InceptionResNetV2;
from keras.applications.vgg16 import preprocess_input;
from tensorflow.keras.applications.vgg16 import decode_predictions;
from tensorflow.keras.callbacks import EarlyStopping;

import os;
from os import listdir;
from PIL import Image as PImage;
import cv2

In [ ]:
from tqdm import tqdm

We have imported the libraries. 

Its Time to export the data

# Data

In [ ]:
train = pd.read_csv('../input/identify-snake-breed-hackerearth/dataset/train.csv');
test = pd.read_csv('../input/identify-snake-breed-hackerearth/dataset/test.csv')

# How many types of breeds are there?

In [ ]:
train['breed'].unique()

In [ ]:
train.head()

# X

We have images in train folder. And, we have name of those breeds in the csv format. We have to merge between these two to make the machine understand which image belongs to which breed.

Lets run a simple for loop

In [ ]:
img_width = 150;
img_height = 150;
x=[];

for i in tqdm (range(train.shape[0])):
    path = '../input/identify-snake-breed-hackerearth/dataset/train/' + train['image_id'][i] + '.jpg'
    img = image.load_img(path, target_size = (img_width, img_height, 3))
    img = image.img_to_array(img)
    img = img/255.0
    x.append(img)
    
#x = np.array(x);
#rm path

Nice, the images are extracted. We have got our x. Now we need y

In [ ]:
 y = train.drop(columns = ['image_id'], axis = 1)

In [ ]:
y = pd.get_dummies(y)

In [ ]:
y = y.to_numpy()

In [ ]:
y

Nice. So, we have converted our y file into numpy array. But how about x? Shouldn't it be a numpy array also?

In [ ]:
x = np.array(x)

# Simple Train Test Split

In [ ]:
mtrain, mtest, ntrain, ntest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
mtrain.shape, ntrain.shape

# Deep Model

In [ ]:
mod = keras.models.Sequential([
                        keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = mtrain[0].shape),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.30),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.30),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.40),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.50),
    
                        keras.layers.Flatten(),
     
                        keras.layers.Dense(units = 128, activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dropout(0.50),
    
                        keras.layers.Dense(units = 128, activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dropout(0.50),
    
    
                        keras.layers.Dense(units = 35, activation = 'softmax')
])
                        

In [ ]:
mod.compile(optimizer= 'Adam' , loss = 'categorical_crossentropy' , metrics= ['accuracy'])

# Training

In [ ]:
mod.fit(mtrain, ntrain, epochs= 500, batch_size = 64, validation_data = (mtest, ntest))

# Test Data

In [ ]:
img_width = 150;
img_height = 150;
z=[];

for i in tqdm (range(test.shape[0])):
    pathe = '../input/identify-snake-breed-hackerearth/dataset/test/' + test['image_id'][i] + '.jpg'
    img1 = image.load_img(pathe, target_size = (img_width, img_height, 3))
    img1 = image.img_to_array(img1)
    img1 = img1/255.0
    #img1 = img1.reshape(1,img_width, img_height, 3 )
    z.append(img1)

In [ ]:
z

Let's convert it into a numpy array

In [ ]:
z = np.array(z)

In [ ]:
z

# Pridiction

In [ ]:
res = mod.predict(z)

In [ ]:
res = np.array(res)

# Getting the Submission File Ready

In [ ]:
cl = train['breed'].unique()

In [ ]:
c = res.argmax(axis = 1);
c.shape

In [ ]:
ped = [cl [index] for index in c];
hehe = pd.DataFrame({'breed': ped})

In [ ]:
hehe.set_index('breed').to_csv('submissionnew.csv')